In [1]:
# Script Name: Combination of Temperature and Energy Demand
# Author: Rahul Kumar
# Date: 4/3/22
# Description: The purpose of the script is to combine the temperature and energy demand data into a unified dataset that 
# can be used to work with

import pandas as pd
import numpy as np
from math import sqrt
import seaborn as sns

# This actually sets the pandas display to show all rows and columns 
# when you are showing a dataframe, without skipping the center
pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)

In [2]:
filepath = '../../H06-2021/' # If you want to run the code, change the file path to where you store the files. This reference
                                # is to where I store the files outside the github clone
    
outpath = '../data/'
# filename = 'totaldemand_nsw'
ext = '.csv'

states = ['nsw','qld','sa','vic']

In [3]:
# 'temprature', 'totaldemand', 'forecastdemand' <-correct your NSW temperature filename to temprature

filename = 'temprature'
temp = pd.DataFrame(columns=['LOCATION','DATETIME','TEMPERATURE'])

for state in states:
    input = pd.read_csv(filepath+filename+'_'+state+ext)
    temp = pd.concat([temp,input], ignore_index=True)
    
temp.to_csv(outpath+filename+ext)
temp = temp[['LOCATION','DATETIME','TEMPERATURE']]
temp.head(15)

,LOCATION,DATETIME,TEMPERATURE
0,Bankstown,1/1/2010 0:00,23.1
1,Bankstown,1/1/2010 0:01,23.1
2,Bankstown,1/1/2010 0:30,22.9
3,Bankstown,1/1/2010 0:50,22.7
4,Bankstown,1/1/2010 1:00,22.6
5,Bankstown,1/1/2010 1:30,22.5
6,Bankstown,1/1/2010 2:00,22.5
7,Bankstown,1/1/2010 2:30,22.4
8,Bankstown,1/1/2010 3:00,22.3
9,Bankstown,1/1/2010 3:30,22.3


In [4]:
filename = 'totaldemand'
demand = pd.DataFrame(columns=['DATETIME','TOTALDEMAND','REGIONID'])

for state in states:
    input = pd.read_csv(filepath+filename+'_'+state+ext)
    demand = pd.concat([demand,input], ignore_index=True)
    
demand.to_csv(outpath+filename+ext)

demand.head(15)

,DATETIME,TOTALDEMAND,REGIONID
0,1/1/2010 0:00,8038.00,NSW1
1,1/1/2010 0:30,7809.31,NSW1
2,1/1/2010 1:00,7483.69,NSW1
3,1/1/2010 1:30,7117.23,NSW1
4,1/1/2010 2:00,6812.03,NSW1
5,1/1/2010 2:30,6544.33,NSW1
6,1/1/2010 3:00,6377.32,NSW1
7,1/1/2010 3:30,6282.85,NSW1
8,1/1/2010 4:00,6211.49,NSW1
9,1/1/2010 4:30,6248.31,NSW1


In [6]:
filename = 'combined'
combined = temp.merge(demand, left_on='DATETIME', right_on = 'DATETIME')

combined.to_csv(outpath+filename+ext)
combined.head(15)

,LOCATION,DATETIME,TEMPERATURE,TOTALDEMAND,REGIONID
0,Bankstown,1/1/2010 0:00,23.1,8038.00,NSW1
1,Bankstown,1/1/2010 0:30,22.9,7809.31,NSW1
2,Bankstown,1/1/2010 1:00,22.6,7483.69,NSW1
3,Bankstown,1/1/2010 1:30,22.5,7117.23,NSW1
4,Bankstown,1/1/2010 2:00,22.5,6812.03,NSW1
5,Bankstown,1/1/2010 2:30,22.4,6544.33,NSW1
6,Bankstown,1/1/2010 3:00,22.3,6377.32,NSW1
7,Bankstown,1/1/2010 3:30,22.3,6282.85,NSW1
8,Bankstown,1/1/2010 4:00,22.1,6211.49,NSW1
9,Bankstown,1/1/2010 4:30,22.2,6248.31,NSW1
